In [ ]:
import util
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from torch.cuda import is_available
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC # dataset is too large, we will not train with whole data
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pycontractions import Contractions

In [ ]:
cont = Contractions(api_key="glove-twitter-100")

# optional, prevents loading on first expand_texts call
cont.load_models()

In [ ]:
dataset = util.load_dataset()

In [ ]:
dataset['text'] = dataset['text'].apply(util.clear_text)
dataset['text'] = list(cont.expand_texts(dataset['text']))
dataset['text'] = dataset['text'].apply(util.lemmatize)
dataset.to_csv('./dataset/clear_lemmatize_expand_texts.csv', index=False)

In [ ]:
dataset = pd.read_csv('./dataset/clear_lemmatize_expand_texts.csv')

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_vectorizer.fit(dataset['text'])
print(tfidf_vectorizer.transform([dataset.iloc[-1]['text']]).toarray().shape)

In [ ]:
cv = CountVectorizer()
cv.fit(dataset['text'])
print(cv.transform([dataset.iloc[-1]['text']]).toarray().shape)

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2", device='cuda' if is_available() else 'cpu')
model.device

In [7]:
dataset['text'] = dataset['text'].astype(str)

In [15]:
embedding = model.encode(dataset['text'], show_progress_bar=True, batch_size=512)
np.save('./dataset/clear_lemmatize_expand_texts_embedding.npy', embedding)

Batches: 100%|██████████| 3125/3125 [14:07<00:00,  3.69it/s] 


In [ ]:
embedding = np.load('./dataset/raw_text_embedding.npy')

In [16]:
X, Y = embedding, dataset['target'].to_numpy()

In [17]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=30, random_state=2024)

In [ ]:
model = SVC(random_state=2024)

In [ ]:
model.fit(x_train[:10000], y_train[:10000])
print(model.score(x_train[:10000], y_train[:10000]))
print(model.score(x_test[:1000], y_test[:1000]))

In [18]:
model = LinearSVC(random_state=2024)

In [ ]:
x_train.shape

In [19]:
model.fit(x_train, y_train)
print(model.score(x_train, y_train))
print(model.score(x_test[:1000], y_test[:1000]))

c:\program_code\tweets_sentiment_analysis\venv\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


0.7737895085532853
0.7333333333333333
